In [ ]:
# Importing necessory libraries
import os
import shutil
import numpy as np
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import  plot_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline


In [ ]:
# Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Where all dataset is there (input images folder copy on dive)
data_dir = '/content/drive/MyDrive/Datasets/flowers' # Google dive path where this folder is created # folder has subfolders for eac class

# Training data dir
training_dir = '/content/drive/MyDrive/Datasets/flowers/train'

# Test data dir
testing_dir = '/content/drive/MyDrive/Datasets/flowers/test'

# Ratio of training and testing data
train_test_ratio = 0.8


def split_dataset_into_test_and_train_sets(all_data_dir = data_dir, training_data_dir = training_dir, testing_data_dir=testing_dir, train_test_ratio = 0.8):
    # Recreate testing and training directories

    if not os.path.exists(training_data_dir): # This '/content/drive/MyDrive/Datasets/flowers/train'
            os.mkdir(training_data_dir)       # Creates this folder '/content/drive/MyDrive/Datasets/flowers/train'

    if not os.path.exists(testing_data_dir):  # This '/content/drive/MyDrive/Datasets/flowers/test'
            os.mkdir(testing_data_dir)        # Creates this folder '/content/drive/MyDrive/Datasets/flowers/test'

    num_training_files = 0
    num_testing_files = 0


    for subdirs, dirs, files in os.walk(all_data_dir):

        category_name = os.path.basename(subdirs)

        # print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
              continue
        if category_name == "train":
              break
        if category_name == "test":
              break

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name

        # creating subdir for each sub category
        if not os.path.exists(training_data_category_dir):  # This '/content/drive/MyDrive/Datasets/flowers/train/category_name'
            os.mkdir(training_data_category_dir)            # Creates this folder '/content/drive/MyDrive/Datasets/flowers/train/category_name'

        if not os.path.exists(testing_data_category_dir):   # This '/content/drive/MyDrive/Datasets/flowers/test/category_name'
            os.mkdir(testing_data_category_dir)             # Creates this folder '/content/drive/MyDrive/Datasets/flowers/test/category_name'

        file_list = glob.glob(os.path.join(subdirs,'*.jpg'))

        #print(os.path.join(all_data_dir, sdir))
        print(str(category_name) + ' has ' + str(len(files)) + ' images')
        random_set = np.random.permutation((file_list))
        # copy percentage of data from each category to train and test directory
        train_list = random_set[:round(len(random_set)*(train_test_ratio))]
        test_list = random_set[-round(len(random_set)*(1-train_test_ratio)):]



        for lists in train_list :
            shutil.copy(lists, training_data_dir + '/' + category_name + '/' )
            num_training_files += 1

        for lists in test_list :
            shutil.copy(lists, testing_data_dir + '/' + category_name + '/' )
            num_testing_files += 1

        if subdirs == "train" or subdirs == "test":
              break


    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [ ]:
split_dataset_into_test_and_train_sets()

daisy has 764 images
dandelion has 1052 images
rose has 784 images
sunflower has 733 images
tulip has 984 images
Processed 3453 training files.
Processed 864 testing files.


In [ ]:
#Building the model from resnet50 pretrained model

# Number of classes in dataset
num_classes = 5

def get_model():
    # Get base model
    # Here we are using ResNet50 as base model
    base_model = ResNet50(weights='imagenet', include_top=False)

    # As we are using ResNet model only for feature extraction and not adjusting the weights
    # we freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = False

    # Get base model output
    base_model_ouput = base_model.output

    # Adding our own layer
    x = GlobalAveragePooling2D()(base_model_ouput)
    # Adding fully connected layer
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)

    model = Model(inputs=base_model.input, outputs=x)
    return model

In [ ]:
# Get the model
model = get_model()
# Compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Summary of model
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, None, 3)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, None, None, 3)  │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, None, None, 64) │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, None, None, 64) │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, None, None, 64) │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, None, None, 64) │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, None, None, 64) │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, None, None, 64) │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, None, None, 64) │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, None, None, 64) │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, None, None, 64) │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, None, None, 64) │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, None, None, 64) │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, None, None,     │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │ 256)                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, None, None,     │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │ 256)                   │                │                        │
├──────────────────────

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64

train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

Found 3453 images belonging to 5 classes.
Found 864 images belonging to 5 classes.


In [ ]:
# Training the fully conncected layer for initial epochs
epochs = 5

# Training the model

model.fit(
    train_generator,
    # steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    # validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

Epoch 1/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 939s 17s/step - accuracy: 0.8940 - loss: 0.3147 - val_accuracy: 0.8889 - val_loss: 0.3432
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 984s 18s/step - accuracy: 0.9075 - loss: 0.2772 - val_accuracy: 0.8877 - val_loss: 0.3369
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 939s 17s/step - accuracy: 0.9077 - loss: 0.2696 - val_accuracy: 0.8877 - val_loss: 0.3339
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 902s 17s/step - accuracy: 0.9136 - loss: 0.2469 - val_accuracy: 0.8924 - val_loss: 0.3194
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 943s 17s/step - accuracy: 0.9213 - loss: 0.2440 - val_accuracy: 0.8854 - val_loss: 0.3491


**Building another model using Resnet50 pretrained model by Freezing the first 140 Layers**

In [ ]:
from tensorflow.keras.optimizers.legacy import SGD

In [ ]:
# More fine tuning the model
# Training the model after 150 layers
# Generally ResNet is good at extracting lower level features so we are not fine tuning initial layers
epochs = 10

split_at = 140
for layer in model.layers[:split_at]:
  layer.trainable = False
for layer in model.layers[split_at:]:
  layer.trainable = True

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate, if we are fine tuning the initial layers
sgd = optimizers.legacy.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)
# What is the difference between model.fit & model.fit_generator
# where image processing is doen (resizing and normalization)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
<ipython-input-12-2c83bf5ad639>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
53/53 [==============================] - 63s 1s/step - loss: 0.3546 - accuracy: 0.8908 - val_loss: 0.3390 - val_accuracy: 0.8774
Epoch 2/10
53/53 [==============================] - 57s 1s/step - loss: 0.2363 - accuracy: 0.9283 - val_loss: 0.2976 - val_accuracy: 0.8906
Epoch 3/10
53/53 [==============================] - 56s 1s/step - loss: 0.1834 - accuracy: 0.9439 - val_loss: 0.2708 - val_accuracy: 0.8966
Epoch 4/10
53/53 [==============================] - 56s 1s/step - loss: 0.1395 - accuracy: 0.9578 - val_loss: 0.2552 - val_accuracy: 0.9050
Epoch 5/10
53/53 [==============================] - 57s 1s/step - loss: 0.1131 - accuracy: 0.9672 - val_loss: 0.2513 - val_accuracy: 0.9099
Epoch 6/10
53/53 [==============================] - 56s 1s/step - loss: 0.0881 - accuracy: 0.9785 - val_loss: 0.2571 - val_accuracy: 0.9135
Epoch 7/10
53/53 [==============================] - 58s 1s/step - loss: 0.0770 - accuracy: 0.9802 - val_loss: 0.2460 - val_accuracy: 0.9159
Epoch 8/10
53/53 [==

There is improved accuracy as compared to previous model where all layers except last classification layer was freezed